In [ ]:
import json
import pandas as pd
import re
import os

In [ ]:
def extract_characters_from_text(text, master_character_list):
    found_characters = set()
    normalized_text = text.lower()

    for char in master_character_list:
        escaped_name = re.escape(char)
        if re.search(r'\b' + escaped_name + r'\b', normalized_text, re.IGNORECASE):
            found_characters.add(char)
            
    return list(found_characters)